GDD Classify Pipeline

0: Install all required packages

In [ ]:
from notebook_ui import install_requirements
install_requirements()

1. Step Set the run configuration.
Be sure to set java memory not over your usable RAM
If Metanome fails, be sure to use JAVA Version 18.0.2.1

In [ ]:
from notebook_ui import config_editor
config_editor()


2. Upload a CSV file or a ZIP archive locally. ZIPs will be extracted automatically.

In [ ]:
from notebook_ui import upload_widget
import json

# Load your config
with open("config.json", "r") as f:
    config = json.load(f)

# Display widget
upload_widget(config)

3. Run the classification pipeline on the dataset:

In [ ]:
%run test_pipeline.py